Output URI
s3://groups-bucket-dblab-905418150721/group23/outputs/query5/

Nα υπολογιστεί, ανά αστυνομικό τμήμα, ο αριθμός εγκλημάτων που έλαβαν χώρα πλησιέστερα σε
αυτό, καθώς και η μέση απόστασή του από τις τοποθεσίες όπου σημειώθηκαν τα συγκεκριμένα περιστατικά. Τα αποτελέσματα να εμφανιστούν ταξινομημένα κατά αριθμό περιστατικών, με φθίνουσα
σειρά (δείτε παράδειγμα στον Πίνακα 4).

Δημιουργούμε το configuration

In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "2",
        "spark.executor.cores": "4",
        "spark.executor.memory": "8g",
        "spark.driver.memory": "2g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2601,application_1732639283265_2560,pyspark,idle,Link,Link,None,
2704,application_1732639283265_2663,pyspark,idle,Link,Link,None,
2706,application_1732639283265_2665,pyspark,idle,Link,Link,None,
2709,application_1732639283265_2668,pyspark,idle,Link,Link,None,


Το query μας τρεχει ως εξής:
* Ενώνουμε τους 2 πίνακες με τα εγκλήματα σε έναν
* Για κάθε έγκλημα, κρατάμε τις συντεταγμένες του μαζί με ένα αναγνωριστικό για distinction από τα άλλα
* Για κάθε αστυνομικό τμήμα κρατάμε το όνομα και τις συντεταγμένες του
* Υπολογίζουμε όλες τις αποστάσεις από κάθε έγκλημα σε κάθε αστυνομικό τμήμα με cross join
* Για τα records με ίδιο crime id τα ταξινομούμε με αύξουσα σειρά απόστασης και κρατάμε τη σειρά
* Κρατάμε μόνο τα records με σειρά 1 (πιο κοντινό)
* Εκτελούμε group by το αστυνομικό τμήμα και για όλα τα εγκλήματα μετράμε τον αριθμό τους και την μέση απόσταση

In [2]:
from pyspark.sql import SparkSession
from sedona.spark import *
from pyspark.sql.functions import col
import time

# Create spark Session
spark = SparkSession \
    .builder \
    .appName("QUERY 5") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)



crime19path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime20path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
stationpath = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv"

start_time = time.time() 

crime19 = spark.read.format('csv').options(header='true').load(crime19path)
crime20 = spark.read.format('csv').options(header='true').load(crime20path)
station = spark.read.format('csv').options(header='true').load(stationpath)


crime19 = crime19.filter((col("LAT").isNotNull()) & (col("LAT") != 0) & (col("LON").isNotNull()) & (col("LON") != 0))

crime20 = crime20.filter((col("LAT").isNotNull()) & (col("LAT") != 0) & (col("LON").isNotNull()) & (col("LON") != 0))



crime19.createOrReplaceTempView("crime19")
crime20.createOrReplaceTempView("crime20")
station.createOrReplaceTempView("station")

query = """
        WITH crimes AS (
            SELECT * FROM crime19
            UNION
            SELECT * FROM crime20
        ), 
        crime_loc AS (
            SELECT ST_Point(LON, LAT) AS geometry, ROW_NUMBER() OVER (ORDER BY LON) AS key
            FROM crimes
        ),
        station_loc AS (
            SELECT DIVISION, ST_POINT(X,Y) as geometry 
            FROM station
        ),
        distances AS (
            SELECT 
                s.DIVISION, 
                ST_DistanceSphere(c.geometry, s.geometry)/1000 AS distance, 
                c.key
            FROM crime_loc c
            CROSS JOIN station_loc s
        ),
        ranked_distances AS (
            SELECT 
                DIVISION,
                distance,
                ROW_NUMBER() OVER (PARTITION BY key ORDER BY distance ASC) AS rank
            FROM distances
        ),
        crime_with_station AS (
            SELECT
                DIVISION,
                distance
            FROM ranked_distances
            WHERE rank = 1
        )
        SELECT 
            DIVISION AS division,
            AVG(distance) AS average_distance,
            COUNT(*) AS `#`
        FROM crime_with_station
        GROUP BY DIVISION
        ORDER BY `#` DESC;
        """



result = spark.sql(query)
result.show()

elapsed_time = time.time() - start_time
print(f"Execution Time : {elapsed_time:.2f} seconds\n")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2714,application_1732639283265_2673,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+------+
|        division|  average_distance|     #|
+----------------+------------------+------+
|       HOLLYWOOD|2.0762639601779624|224340|
|        VAN NUYS|2.9533697428196763|210134|
|       SOUTHWEST|2.1913988057806324|188901|
|        WILSHIRE|2.5926655329788733|185996|
|     77TH STREET| 1.716544971969946|171827|
|         OLYMPIC|1.7236036971777262|170897|
| NORTH HOLLYWOOD|2.6430060941418145|167854|
|         PACIFIC| 3.850070655307948|161359|
|         CENTRAL|0.9924764374568322|153871|
|         RAMPART|1.5345341879188912|152736|
|       SOUTHEAST|2.4218662158882456|152176|
|     WEST VALLEY|3.0356712163142756|138643|
|         TOPANGA|  3.29695484175609|138217|
|        FOOTHILL| 4.250921708424341|134896|
|          HARBOR| 3.702561599356797|126747|
|      HOLLENBECK| 2.680181237706614|115837|
|WEST LOS ANGELES|2.7924572890339463|115781|
|          NEWTON|1.6346357397097593|111110|
|       NORTHEAST|  3.62366552460384|108109|
|         

In [3]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.cores": "2",
        "spark.executor.memory": "4g", 
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2717,application_1732639283265_2676,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2601,application_1732639283265_2560,pyspark,idle,Link,Link,None,
2704,application_1732639283265_2663,pyspark,idle,Link,Link,None,
2706,application_1732639283265_2665,pyspark,idle,Link,Link,None,
2715,application_1732639283265_2674,pyspark,busy,Link,Link,None,
2716,application_1732639283265_2675,pyspark,idle,Link,Link,None,
2717,application_1732639283265_2676,pyspark,idle,Link,Link,None,✔


In [4]:
from pyspark.sql import SparkSession
from sedona.spark import *
from pyspark.sql.functions import col
import time

# Create spark Session
spark = SparkSession \
    .builder \
    .appName("QUERY 5") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)



crime19path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime20path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
stationpath = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv"

start_time = time.time() 

crime19 = spark.read.format('csv').options(header='true').load(crime19path)
crime20 = spark.read.format('csv').options(header='true').load(crime20path)
station = spark.read.format('csv').options(header='true').load(stationpath)


crime19 = crime19.filter((col("LAT").isNotNull()) & (col("LAT") != 0) & (col("LON").isNotNull()) & (col("LON") != 0))

crime20 = crime20.filter((col("LAT").isNotNull()) & (col("LAT") != 0) & (col("LON").isNotNull()) & (col("LON") != 0))



crime19.createOrReplaceTempView("crime19")
crime20.createOrReplaceTempView("crime20")
station.createOrReplaceTempView("station")

query = """
        WITH crimes AS (
            SELECT * FROM crime19
            UNION
            SELECT * FROM crime20
        ), 
        crime_loc AS (
            SELECT ST_Point(LON, LAT) AS geometry, ROW_NUMBER() OVER (ORDER BY LON) AS key
            FROM crimes
        ),
        station_loc AS (
            SELECT DIVISION, ST_POINT(X,Y) as geometry 
            FROM station
        ),
        distances AS (
            SELECT 
                s.DIVISION, 
                ST_DistanceSphere(c.geometry, s.geometry)/1000 AS distance, 
                c.key
            FROM crime_loc c
            CROSS JOIN station_loc s
        ),
        ranked_distances AS (
            SELECT 
                DIVISION,
                distance,
                ROW_NUMBER() OVER (PARTITION BY key ORDER BY distance ASC) AS rank
            FROM distances
        ),
        crime_with_station AS (
            SELECT
                DIVISION,
                distance
            FROM ranked_distances
            WHERE rank = 1
        )
        SELECT 
            DIVISION AS division,
            AVG(distance) AS average_distance,
            COUNT(*) AS `#`
        FROM crime_with_station
        GROUP BY DIVISION
        ORDER BY `#` DESC;
        """



result = spark.sql(query)
result.show()

elapsed_time = time.time() - start_time
print(f"Execution Time : {elapsed_time:.2f} seconds\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+------+
|        division|  average_distance|     #|
+----------------+------------------+------+
|       HOLLYWOOD|  2.07626396017796|224340|
|        VAN NUYS|2.9533697428196746|210134|
|       SOUTHWEST| 2.191398805780634|188901|
|        WILSHIRE| 2.592665532978874|185996|
|     77TH STREET|1.7165449719699533|171827|
|         OLYMPIC|1.7236036971777253|170897|
| NORTH HOLLYWOOD| 2.643006094141829|167854|
|         PACIFIC|3.8500706553079516|161359|
|         CENTRAL|0.9924764374568322|153871|
|         RAMPART|1.5345341879188898|152736|
|       SOUTHEAST|2.4218662158882505|152176|
|     WEST VALLEY| 3.035671216314276|138643|
|         TOPANGA|3.2969548417560897|138217|
|        FOOTHILL| 4.250921708424342|134896|
|          HARBOR| 3.702561599356801|126747|
|      HOLLENBECK|2.6801812377066114|115837|
|WEST LOS ANGELES| 2.792457289033946|115781|
|          NEWTON|1.6346357397097588|111110|
|       NORTHEAST|3.6236655246038407|108109|
|         

In [5]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "8",
        "spark.executor.cores": "1",
        "spark.executor.memory": "2g",
        "spark.driver.memory": "2g"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2719,application_1732639283265_2678,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2601,application_1732639283265_2560,pyspark,idle,Link,Link,None,
2704,application_1732639283265_2663,pyspark,idle,Link,Link,None,
2706,application_1732639283265_2665,pyspark,idle,Link,Link,None,
2715,application_1732639283265_2674,pyspark,busy,Link,Link,None,
2718,application_1732639283265_2677,pyspark,idle,Link,Link,None,
2719,application_1732639283265_2678,pyspark,idle,Link,Link,None,✔


In [6]:
from pyspark.sql import SparkSession
from sedona.spark import *
from pyspark.sql.functions import col
import time

# Create spark Session
spark = SparkSession \
    .builder \
    .appName("QUERY 5") \
    .getOrCreate()

# Create sedona context
sedona = SedonaContext.create(spark)



crime19path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime20path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"
stationpath = "s3://initial-notebook-data-bucket-dblab-905418150721/LA_Police_Stations.csv"

start_time = time.time() 

crime19 = spark.read.format('csv').options(header='true').load(crime19path)
crime20 = spark.read.format('csv').options(header='true').load(crime20path)
station = spark.read.format('csv').options(header='true').load(stationpath)


crime19 = crime19.filter((col("LAT").isNotNull()) & (col("LAT") != 0) & (col("LON").isNotNull()) & (col("LON") != 0))

crime20 = crime20.filter((col("LAT").isNotNull()) & (col("LAT") != 0) & (col("LON").isNotNull()) & (col("LON") != 0))



crime19.createOrReplaceTempView("crime19")
crime20.createOrReplaceTempView("crime20")
station.createOrReplaceTempView("station")

query = """
        WITH crimes AS (
            SELECT * FROM crime19
            UNION
            SELECT * FROM crime20
        ), 
        crime_loc AS (
            SELECT ST_Point(LON, LAT) AS geometry, ROW_NUMBER() OVER (ORDER BY LON) AS key
            FROM crimes
        ),
        station_loc AS (
            SELECT DIVISION, ST_POINT(X,Y) as geometry 
            FROM station
        ),
        distances AS (
            SELECT 
                s.DIVISION, 
                ST_DistanceSphere(c.geometry, s.geometry)/1000 AS distance, 
                c.key
            FROM crime_loc c
            CROSS JOIN station_loc s
        ),
        ranked_distances AS (
            SELECT 
                DIVISION,
                distance,
                ROW_NUMBER() OVER (PARTITION BY key ORDER BY distance ASC) AS rank
            FROM distances
        ),
        crime_with_station AS (
            SELECT
                DIVISION,
                distance
            FROM ranked_distances
            WHERE rank = 1
        )
        SELECT 
            DIVISION AS division,
            AVG(distance) AS average_distance,
            COUNT(*) AS `#`
        FROM crime_with_station
        GROUP BY DIVISION
        ORDER BY `#` DESC;
        """



result = spark.sql(query)
result.show()

elapsed_time = time.time() - start_time
print(f"Execution Time : {elapsed_time:.2f} seconds\n")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+------------------+------+
|        division|  average_distance|     #|
+----------------+------------------+------+
|       HOLLYWOOD|2.0762639601779616|224340|
|        VAN NUYS|2.9533697428196706|210134|
|       SOUTHWEST|2.1913988057806324|188901|
|        WILSHIRE| 2.592665532978874|185996|
|     77TH STREET| 1.716544971969952|171827|
|         OLYMPIC|1.7236036971777255|170897|
| NORTH HOLLYWOOD| 2.643006094141818|167854|
|         PACIFIC| 3.850070655307949|161359|
|         CENTRAL|0.9924764374568318|153871|
|         RAMPART|1.5345341879188896|152736|
|       SOUTHEAST|2.4218662158882487|152176|
|     WEST VALLEY|3.0356712163142756|138643|
|         TOPANGA|3.2969548417560905|138217|
|        FOOTHILL| 4.250921708424341|134896|
|          HARBOR| 3.702561599356798|126747|
|      HOLLENBECK|2.6801812377066145|115837|
|WEST LOS ANGELES|2.7924572890339467|115781|
|          NEWTON|1.6346357397097584|111110|
|       NORTHEAST|3.6236655246038407|108109|
|         